# Importujeme numpy

Chceme-li použít `numpy`, je samozřejmě nutné modul importovat. Obvykle se použivá zkratka `np`: 


In [ ]:
import numpy as np


# Numpy a práce se soubory

### ASCII soubory
S textovými (ASCII) soubory obsahující data se setkáváme stále často, přestože to z mnoha důvodů není ideální formát. Na čtení ASCII (spadá sem i CSV) máme v Numpy `genfromtxt` a `loadtxt`. V [dokumentaci](http://docs.scipy.org/doc/numpy/reference/routines.io.html) se dozvíte, jak přesně fungují a jaké mají argumenty.

Pomocí `%%file` vytvoříme soubor `ascii_data_1.txt`

In [ ]:
%%file ascii_data_1.txt
1 -6.1 -6.1 -6.1 1
2 -15.4 -15.4 -15.4 1
3 -15.0 -15.0 -15.0 1
4 -19.3 -19.3 -19.3 1
5 -16.8 -16.8 -16.8 1
6 -11.4 -11.4 -11.4 1
7 -7.6 -7.6 -7.6 1
8 -7.1 -7.1 -7.1 1
9 -10.1 -10.1 -10.1 1
10 -9.5 -9.5 -9.5 1


Nyní se pokusíme soubor načíst pomocí `genfromtxt`.

In [ ]:
data = np.genfromtxt('ascii_data_1.txt')
print(data)


`loadtxt` by mělo fungovat také:

In [ ]:
data = np.loadtxt('ascii_data_1.txt')
print(data)

`savetxt` můžeme použít na uložení
- parametr `fmt` určuje formát, který se použije pro uložení dat
 - např. `fmt="%.2f"` uloží data s 2 desetinnými místy
 - např. `fmt="%6g"` uloží data s prázdnými místy doplněnými nulami na 6 míst

In [ ]:
np.savetxt("ascii_data_1_new.txt", np.random.rand(5,3), fmt="%.2f")
np.savetxt("ascii_data_2_new.txt", data, fmt="%6g")


Soubor můžeme vypsat:

In [ ]:
%less ascii_data_1_new.txt


In [ ]:
%less ascii_data_2_new.txt


Obecně se snažte textovým souborům (včetně csv apod.) pro numerická data vyhýbat. Jejich formát je vždy do značné míry neurčitý a na disku zabírají zbytečně moc místa. Výhodou je pouze jednoduchost zobrazení v textovém editoru nebo příkazové řadce.

### Binární formáty

Pro numerická data se daleko více hodí binární soubory, které jsou dobře definované a úsporné na místo. Pokud použijeme vhodný a rozšířený formát, nemusíme se bát ani přenositelnosti.

Numpy má vlastní NPY formát. Ten je pochopitelně jednoduchý na používání v NumPy, s přenositelností (pro neuživatele Pythonu a obecně další systémy) je to ale už horší. Pomocí `save` a `load` můžete jednoduše ukládat a nahrávat Numpy objekty.

In [ ]:
np.save("binary_data_1_new.npy", np.random.rand(5,3))


In [ ]:
data = np.load("binary_data_1_new.npy")
print(data)

In [ ]:
# pojďmě porovnat kolik místa zabírají jednotlivé soubory
data = np.random.rand(100, 100)
np.save("binary_data_1_new.npy", data)
np.savetxt("ascii_data_1_new.txt", data)
%ls -l

## HDF5 formát
Velice dobrým a rozšířeným standardem je pak [HDF5](https://www.hdfgroup.org/solutions/hdf5/). Pro Python je jednoduché tento foromát používat pomocí knihovny [h5py](https://www.h5py.org/).

In [ ]:
# pokud nemáte h5py nainstalované, můžete jednoduše nainstalovat přímo z notebooku momocí
# %conda install h5py
# nebo pokud používáte pip prostředí
# %pip install h5py


In [ ]:
# můžete instalovat přímo z notebooku, stačí odstranit komentář
# !pip install h5py -U

In [ ]:
import h5py


V HDF5 souborech jsou data ve stromové struktuře (obdoba aresářů a souborů). Soubor se dvěma datasety můžeme vytvořit např. takto:

In [ ]:
data_pole = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.int32)
data_binary = np.array([True, False, True, False, True, False], dtype=bool)
with h5py.File("test_hdf5.h5", "w") as hdf5_file:
    hdf5_file.create_dataset("data1", data=data_pole)
    hdf5_file.create_dataset("data_nahodne", data=np.random.random_sample((3, 4)))
    hdf5_file.create_dataset("data_binarni", data=data_binary)


In [ ]:
with h5py.File("test_hdf5.h5", "r") as hdf5_file:
    data_hdf5 = hdf5_file["data1"]
    print(data_hdf5)
    # musíme data "nahrát" pomocí [:], jinak by byl výsledek jen "ukazatel" na data
    data_pole = hdf5_file["data1"][:]
    data_binary = hdf5_file["data_binarni"][:]
    data_nahodne = hdf5_file["data_nahodne"][:]



In [ ]:
# "ukazatel" na data přestane fungovat, pokud soubor zavřeme
data_hdf5


Jelikož v HDF5 souboru může být velké množství dat (mnoho datasetů, velká data), je čtení dat z HDF5 "lazy": Dokud data opravdu nepotřebujeme v paměti (např. v NumPy poli), data zůstávají jen v souboru a v paměti máme jen jejich popis, jakýsi ukazatel na data.